<a href="https://colab.research.google.com/github/Augusto-Seixas-UFV/seixas-ufv-iac/blob/main/C%C3%B3pia_de_extrator_pdf_ocr_para_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install tesseract-ocr -y
!apt install poppler-utils -y
!pip install pdfplumber pytesseract pdf2image pandas pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (270 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing 

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving fatura Cartão ELO maio-junho.pdf to fatura Cartão ELO maio-junho.pdf


In [ ]:
import pdfplumber
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
import tempfile
import re

def extrair_texto_pdf(path_pdf):
    texto_total = ""
    try:
        with pdfplumber.open(path_pdf) as pdf:
            for pagina in pdf.pages:
                texto_total += pagina.extract_text() + "\n"
        if texto_total.strip():
            return texto_total
        else:
            raise ValueError("PDF sem texto. Partindo para OCR.")
    except:
        return None

def ocr_pdf(path_pdf):
    texto_ocr = ""
    with tempfile.TemporaryDirectory() as path:
        imagens = convert_from_path(path_pdf, dpi=300, output_folder=path)
        for img in imagens:
            texto = pytesseract.image_to_string(img, lang='por')
            texto_ocr += texto + "\n"
    return texto_ocr

def processar_linhas(texto):
    padrao = re.compile(r"(\d{2}/\d{2})\s+(.+?)\s+(-?\d+[\.,]\d{2})")
    dados = []
    for linha in texto.split('\n'):
        linha = linha.strip()
        match = padrao.search(linha)
        if match:
            data, descricao, valor = match.groups()
            valor = valor.replace('.', '').replace(',', '.')
            dados.append((data, descricao, float(valor)))
    return dados

def salvar_csv(dados, nome_saida):
    df = pd.DataFrame(dados, columns=['Data', 'Descrição', 'Valor'])
    df.to_csv(nome_saida, index=False, encoding='utf-8-sig')
    print(f"CSV salvo como: {nome_saida}")

def processar_extrato(path_pdf, nome_csv='extrato_processado.csv'):
    texto = extrair_texto_pdf(path_pdf)
    if texto is None or not texto.strip():
        print("Usando OCR...")
        texto = ocr_pdf(path_pdf)
    dados = processar_linhas(texto)
    if dados:
        salvar_csv(dados, nome_csv)
    else:
        print("Nenhum dado reconhecido. Verifique o layout do extrato.")


In [ ]:

processar_extrato("seuarquivo.pdf")

Usando OCR...


PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file 'seuarquivo.pdf': No such file or directory.


In [ ]:
processar_extrato("fatura Cartão ELO maio-junho.pdf")

Usando OCR...


PDFPageCountError: Unable to get page count.
Command Line Error: Incorrect password


In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import tempfile

def ocr_pdf(path_pdf):
    texto_ocr = ""
    with tempfile.TemporaryDirectory() as path:
        try:
            imagens = convert_from_path(path_pdf, dpi=300, output_folder=path)
        except Exception as e:
            if "Incorrect password" in str(e):
                print("🔒 PDF protegido por senha.")
                senha = input("Digite a senha do PDF: ")
                imagens = convert_from_path(path_pdf, dpi=300, output_folder=path, userpw=senha)
            else:
                raise e
        for img in imagens:
            texto = pytesseract.image_to_string(img, lang='por')
            texto_ocr += texto + "\n"
    return texto_ocr


In [ ]:
processar_extrato("fatura Cartão ELO maio-junho.pdf")


Usando OCR...
🔒 PDF protegido por senha.
Digite a senha do PDF: 07543609894


TesseractError: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/por.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'por\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [ ]:
!apt install tesseract-ocr -y
!apt install tesseract-ocr-por -y
!apt install poppler-utils -y
!pip install pdfplumber pytesseract pdf2image pandas pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-por
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 856 kB of archives.
After this operation, 1,998 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-por all 1:4.00~git30-7274cfa-1.1 [856 kB]
Fetched 856 kB in 0s (2,773 kB/s)
Selecting previously unselected package tesseract-ocr-por.
(Reading database ... 126338 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-por_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-por (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving fatura Cartão ELO maio-junho.pdf to fatura Cartão ELO maio-junho (1).pdf


In [ ]:
nome_arquivo = list(uploaded.keys())[0]
print(f"Arquivo enviado: {nome_arquivo}")


Arquivo enviado: fatura Cartão ELO maio-junho (1).pdf


In [ ]:
processar_extrato("fatura Cartão ELO maio-junho (1).pdf")


Usando OCR...
🔒 PDF protegido por senha.
Digite a senha do PDF: 07543609894
CSV salvo como: extrato_processado.csv


In [ ]:
from google.colab import files
files.download("extrato_processado.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Instala o Tesseract OCR, o motor de reconhecimento óptico de caracteres, o parâmetro ""-y"confirma automaticamente a instalação, sem precisar do "yes"
!apt install tesseract-ocr -y
# Instala o pacote de idioma português (por) para o Tesseract
!apt install tesseract-ocr-por -y
# Instala a ferramentas da biblioteca Poppler, que permite lidar com PDFs no nível de renderização e conversão. Serão utilizadas especificamente o "pdf2image" que depende do utilitário
# "pdftoppm" contido neste pacote.
# Isto é necessário quando o PDF não tem texto embutido (por exemplo, um escaneamento)
!apt install poppler-utils -y
# Instala cinco bibliotecas Python fundamentais para o nosso fluxo.
# pdfplumber que extrai o texto real de PDFs (sem OCR) e que funciona apenas com PDFs gerados digitalmente
# pytesseract é o conector entre o Python e o Tesseract OCR
# pdf2image converte cada página do PDF em uma imagem para o Tesseract poder processar
# pandas organiza os dados extraídos em tabelas (Dataframe) e exporta para ".csv" ou ".xlsx".
# A Biblioteca pandas é a base para análise de dados com Python
# pillow é a biblioteca para trabalhar com imagens em Python (abrir, salvar, converter, etc.)
!pip install pdfplumber pytesseract pdf2image pandas pillow